RAW (MOABB) to CSV

This code convert the data sets from RAW format to CSV format using MOABB.

It has been specifically conceived for BCI data.

This script is for bi2015a-3 


In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# Forward your file with all .csv of the dataset
# Cattan files have been doswloaded from the zenodo repository, we need to clean it before converting to npz
file_dir = "C:\\Users\\doumif\\Downloads\\bi2015a"
output_dir = "C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\P300\\bi2015a-3"
all_files = [os.path.join(file_dir, file) for file in os.listdir(file_dir)]
all_files

In [ ]:
file = "C:\\Users\\doumif\\Downloads\\bi2015a\\subject_01_csv\\subject_01_session_03.csv"

In [ ]:
df = pd.read_csv(file, header=None)
data = np.array(df)

In [ ]:
def process_eeg_file(subject_folder_path, output_dir):
    """
    Process a single subject's EEG file for session 3.


    Parameters:
    -----------
    subject_folder_path : str
        Path to the subject folder (e.g., "bi2015a/subject_XX_csv")
    output_dir : str
        Directory where processed file will be saved


    Returns:
    --------
    str
        Path to the processed file
    """
    # Extract subject number from the subject folder name
    subject_folder = os.path.basename(subject_folder_path)
    subject_num = subject_folder.split('_')[1]  # Extract 'XX' from 'subject_XX_csv'

    # Construct path to the session 1 CSV file within the subject folder
    csv_file_path = os.path.join(subject_folder_path, f"subject_{subject_num}_session_03.csv")

    # Create new filename format (same format, but ensures consistency)
    new_filename = f"subject_{subject_num}_session_03.csv"

    # Read and process the data
    df = pd.read_csv(csv_file_path, header=None)
    data = np.array(df)

    mask_valid_rows = ~np.isnan(data).any(axis=1)
    data = data[mask_valid_rows]

    # rearranging the stim colunm
    data[:, 34] = data[:, 34] * 2

    # Transfer 2s from target column (column 35) to non_target column (column 34)
    mask = data[:, 34] == 2
    data[mask, 33] = 2

    data = np.delete(data, [34], axis=1)

    # Convert back to DataFrame
    df_processed = pd.DataFrame(data)

    # Save processed file
    output_path = os.path.join(output_dir, new_filename)
    df_processed.to_csv(output_path, index=False, header=False)

    print(f"Processed and saved: {new_filename}")
    print(f"Shape: {data.shape}")

    # Adjust the column index if needed after deletion
    # Note: After deleting column 17, the original column 18 becomes 17
    unique_values, counts = np.unique(data[:, 33], return_counts=True)
    for value, count in zip(unique_values, counts):
        print(f"Dans {new_filename} : Nombre de {value} = {count}")

    return output_path


In [ ]:
for i, file in enumerate(all_files, 1):
    print(f"File number {i} out of {len(all_files)}")
    csv = process_eeg_file(file, output_dir)
